In [1]:
import os
import requests
from pystac_client import Client
import dotenv

dotenv.load_dotenv()    

cdse_username = os.environ.get("CDSE_USERNAME")
cdse_password = os.environ.get("CDSE_PASSWORD")

print(cdse_username, cdse_password)



jeff@dl4eo.com !tkK#?yF8Jg8phY9


In [36]:
token_url = 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token'
payload = {
    'client_id': 'cdse-public',
    'grant_type': 'password',
    'username': cdse_username,
    'password': cdse_password
}

response = requests.post(token_url, data=payload)
response.raise_for_status()
access_token = response.json().get('access_token')

In [41]:
import requests
import urllib.parse

# Define search parameters
collection_name = 'SENTINEL-2'
product_type = 'S2MSI1C'
max_cloud_cover = 10  # Maximum cloud cover percentage
aoi_wkt = 'POLYGON((13.0 45.0, 14.0 45.0, 14.0 46.0, 13.0 46.0, 13.0 45.0))'  # WKT format
start_date = '2023-01-01T00:00:00.000Z'
end_date = '2023-01-31T23:59:59.999Z'

# Construct filters
filter_collection = f"Collection/Name eq '{collection_name}'"
filter_product_type = (
    "Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'productType' "
    f"and att/Value eq '{product_type}')"
)
filter_aoi = (
    f"OData.CSC.Intersects(area=geography'SRID=4326;{aoi_wkt}')"
)
filter_dates = (
    f"ContentDate/Start gt {start_date} and "
    f"ContentDate/End lt {end_date}"
)
filter_cloud_cover = (
    "Attributes/OData.CSC.DoubleAttribute/any(att:att/Name eq 'cloudCover' "
    f"and att/OData.CSC.DoubleAttribute/Value le {max_cloud_cover})"
)

# Combine filters
combined_filter = (
    f"{filter_collection} and {filter_product_type} and {filter_dates} and {filter_cloud_cover} and {filter_aoi}"
)

# From documentation:
#combined_filter = 'not (Collection/Name eq \'SENTINEL-2\') and ContentDate/Start gt 2022-05-03T00:00:00.000Z and ContentDate/Start lt 2022-05-03T00:10:00.000Z'

# Encode the combined filter for URL inclusion
encoded_filter = urllib.parse.quote(combined_filter)

# Construct the full query URL
query_url = f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter={encoded_filter}"
print(query_url)

# Set up headers with the access token
headers = {
    'Authorization': f'Bearer {access_token}',
    'Accept': 'application/json'
}

# Execute the query
response = requests.get(query_url, headers=headers)

# Check for HTTP errors
try:
    response.raise_for_status()
    products = response.json().get('value', [])
    print(f"Found {len(products)} products.")
except requests.exceptions.HTTPError as err:
    print(f"HTTP error occurred: {err}")
    print(f"Response content: {response.content.decode()}")
except Exception as err:
    print(f"An error occurred: {err}")

https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=Collection/Name%20eq%20%27SENTINEL-2%27%20and%20Attributes/OData.CSC.StringAttribute/any%28att%3Aatt/Name%20eq%20%27productType%27%20and%20att/Value%20eq%20%27S2MSI1C%27%29%20and%20ContentDate/Start%20gt%202023-01-01T00%3A00%3A00.000Z%20and%20ContentDate/End%20lt%202023-01-31T23%3A59%3A59.999Z%20and%20Attributes/OData.CSC.DoubleAttribute/any%28att%3Aatt/Name%20eq%20%27cloudCover%27%20and%20att/OData.CSC.DoubleAttribute/Value%20le%2010%29%20and%20OData.CSC.Intersects%28area%3Dgeography%27SRID%3D4326%3BPOLYGON%28%2813.0%2045.0%2C%2014.0%2045.0%2C%2014.0%2046.0%2C%2013.0%2046.0%2C%2013.0%2045.0%29%29%27%29
Found 4 products.


In [28]:
import pandas as pd

# Convert the product list to a DataFrame
df = pd.DataFrame(products)
df



,@odata.mediaContentType,Id,Name,ContentType,ContentLength,OriginDate,PublicationDate,ModificationDate,Online,EvictionDate,S3Path,Checksum,ContentDate,Footprint,GeoFootprint
0,application/octet-stream,ff6010a5-4890-5c4a-a62f-d3448d9960d1,S3B_SL_2_LST____20220503T000015_20220503T00031...,application/octet-stream,57084746,2022-05-03T01:40:45.402000Z,2022-05-03T01:45:06.066372Z,2024-07-20T23:30:42.330611Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-3/SLSTR/SL_2_LST/2022/05/03/S...,"[{'Value': 'd75196c2ec29fdbdf9707e8278029173',...","{'Start': '2022-05-03T00:00:15.373000Z', 'End'...",geography'SRID=4326;POLYGON ((-29.4495 -39.747...,"{'type': 'Polygon', 'coordinates': [[[-29.4495..."
1,application/octet-stream,711d4854-6419-5513-9e19-25b166cdbcd5,S3B_SL_1_RBT____20220503T000015_20220503T00031...,application/octet-stream,339040034,2022-05-03T01:40:27.670000Z,2022-05-03T01:41:40.409975Z,2024-07-20T23:30:49.342684Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-3/SLSTR/SL_1_RBT/2022/05/03/S...,"[{'Value': 'e279703cafc604f127e9fd3526389201',...","{'Start': '2022-05-03T00:00:15.373000Z', 'End'...",geography'SRID=4326;POLYGON ((-29.4495 -39.747...,"{'type': 'Polygon', 'coordinates': [[[-29.4495..."
2,application/octet-stream,c6d30f04-e179-582e-82bb-bf8057a8247a,S3B_SL_2_WST____20220503T000015_20220503T00031...,application/octet-stream,22205178,2022-05-03T01:38:51.200000Z,2022-05-03T02:43:42.265498Z,2024-07-20T23:30:40.741793Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-3/SLSTR/SL_2_WST/2022/05/03/S...,"[{'Value': '65c53a591a00dd0753daef05466711a6',...","{'Start': '2022-05-03T00:00:15.373000Z', 'End'...",geography'SRID=4326;POLYGON ((-29.4495 -39.747...,"{'type': 'Polygon', 'coordinates': [[[-29.4495..."
3,application/octet-stream,4a4ef482-84a2-551d-8086-e3de6d39c488,S3B_SL_1_RBT____20220503T000015_20220503T00031...,application/octet-stream,336069884,2022-05-03T13:29:17.527000Z,2022-05-03T13:35:06.464540Z,2024-07-20T23:30:49.323959Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-3/SLSTR/SL_1_RBT/2022/05/03/S...,"[{'Value': '049ec6087a7c41a00213f11db39d2bf5',...","{'Start': '2022-05-03T00:00:15.373000Z', 'End'...",geography'SRID=4326;POLYGON ((-29.448 -39.7539...,"{'type': 'Polygon', 'coordinates': [[[-29.448,..."
4,application/octet-stream,6b19efbf-97d7-5edd-8eb5-ec3c973ee450,S3B_SL_2_FRP____20220503T000015_20220503T00031...,application/octet-stream,70092590,2022-05-03T13:30:35.733000Z,2022-05-03T13:33:10.882892Z,2024-07-20T23:30:42.978470Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-3/SLSTR/SL_2_FRP___/2022/05/0...,"[{'Value': '53e7d5c6088286b1904f8cae764d4e3e',...","{'Start': '2022-05-03T00:00:15.374000Z', 'End'...",geography'SRID=4326;POLYGON ((-29.448 -39.7539...,"{'type': 'Polygon', 'coordinates': [[[-29.448,..."
5,application/octet-stream,48fca349-79a2-58f5-bf22-08c62ed76afb,S3B_SL_2_LST____20220503T000015_20220503T00031...,application/octet-stream,54127536,2022-05-03T23:01:17.780000Z,2022-05-03T23:05:08.459539Z,2024-07-20T23:30:41.950091Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-3/SLSTR/SL_2_LST/2022/05/03/S...,"[{'Value': 'cc2c99759a90726b49d8fac088d1bc0f',...","{'Start': '2022-05-03T00:00:15.374000Z', 'End'...",geography'SRID=4326;POLYGON ((-29.448 -39.7539...,"{'type': 'Polygon', 'coordinates': [[[-29.448,..."
6,application/octet-stream,b9eaa1e7-bdf7-58d5-a04a-d0dc4a2b7a33,S3A_OL_1_EFR____20220503T000040_20220503T00034...,application/octet-stream,0,2022-05-03T02:10:02.480000Z,2022-05-03T02:11:26.884097Z,2022-05-03T02:11:26.884097Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-3/OLCI/OL_1_EFR/2022/05/03/S3...,[],"{'Start': '2022-05-03T00:00:39.649000Z', 'End'...",geography'SRID=4326;POLYGON ((140.299 -0.12728...,"{'type': 'Polygon', 'coordinates': [[[140.299,..."
7,application/octet-stream,8e9b638d-135d-55dd-839f-01218273678e,S3A_OL_2_LFR____20220503T000040_20220503T00034...,application/octet-stream,0,2022-05-03T02:19:31.922000Z,2022-05-03T02:25:15.471238Z,2022-05-03T02:25:15.471238Z,True,999

In [27]:
# Display relevant columns
if not df.empty:
    print(df[['Name', 'Id', 'ContentDate', 'GeoFootprint']])
else:
    print("No products found matching the search criteria.")

                                                 Name  \
0   S3B_SL_2_LST____20220503T000015_20220503T00031...   
1   S3B_SL_1_RBT____20220503T000015_20220503T00031...   
2   S3B_SL_2_WST____20220503T000015_20220503T00031...   
3   S3B_SL_1_RBT____20220503T000015_20220503T00031...   
4   S3B_SL_2_FRP____20220503T000015_20220503T00031...   
5   S3B_SL_2_LST____20220503T000015_20220503T00031...   
6   S3A_OL_1_EFR____20220503T000040_20220503T00034...   
7   S3A_OL_2_LFR____20220503T000040_20220503T00034...   
8   S3A_SL_2_LST____20220503T000040_20220503T00034...   
9   S3A_OL_2_WFR____20220503T000040_20220503T00034...   
10  S3A_SL_2_WST____20220503T000040_20220503T00034...   
11  S3A_SL_1_RBT____20220503T000040_20220503T00034...   
12  S3A_SY_2_SYN____20220503T000040_20220503T00034...   
13  S3A_SL_2_FRP____20220503T000040_20220503T00034...   
14  S3A_SY_2_SYN____20220503T000040_20220503T00034...   
15  S3A_OL_2_LFR____20220503T000040_20220503T00034...   
16  S3A_SL_2_LST____20220503T00

In [3]:
import os
import supabase
import dotenv

dotenv.load_dotenv()   
supabase = supabase.create_client(os.environ.get("SUPABASE_URL"), os.environ.get("SUPABASE_KEY"))



In [8]:
import logging
import asyncio
from typing import Optional

logger = logging.getLogger(__name__)

async def update_cog_status(identifier: str, status: str, bucket: Optional[str] = None, path: Optional[str] = None) -> None:
    """Update the status of a COG in Supabase.

    Args:
        identifier: The Sentinel-2 image identifier.
        status: The status to set ('processing' or 'ready').
        bucket: Optional bucket name where the COG is stored.
        path: Optional path to the COG in the bucket.
    """
    try:
        data = {"identifier": identifier, "status": status}
        if bucket:
            data["bucket"] = bucket
        if path:
            data["path"] = path

        response = (
            supabase.table("sentinel2_cogs")
            .upsert(data, on_conflict="identifier")
            .execute()
        )
        if not response.data:
            logger.error(f"Failed to update COG status for {identifier}")
        else:
            logger.info(f"Updated COG status for {identifier} to {status}")
    except Exception as e:
        logger.error(f"Error updating COG status: {str(e)}")
        raise

In [9]:
await update_cog_status("50b1b57b-3b66-4d55-92eb-c844f6324db5", "ready")